In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from pandas import *
from numpy import *
import jieba as jb
import re

In [2]:
labels = [];datas = [];
fr = open('toutiao_cat_data.txt',encoding='utf-8')
for line in fr.readlines():
    lineArr = line.strip().split("_!_")#strip()删除换行，split("_!_"把每行文本按照_!_分开
    labels.append(lineArr[2])#标签
    datas.append(lineArr[3] + lineArr[4])#特征：标题+新闻关键词
#用DataFrame()存储
df = pandas.DataFrame()
df['data'] = datas
df['label'] = labels

In [3]:
df

,data,label
0,"京城最值得你来场文化之旅的博物馆保利集团,马未都,中国科学技术馆,博物馆,新中国",news_culture
1,发酵床的垫料种类有哪些？哪种更好？,news_culture
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture
4,黄杨木是什么树？,news_culture
...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military
382685,"又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！以色列,普京,俄罗斯,叙利亚,车臣",news_world
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world


In [4]:
#列表中添加类型id
df['id'] = df['label'].factorize()[0]
id_df = df[['label', 'id']].drop_duplicates().sort_values('id').reset_index(drop=True)

In [5]:
df

,data,label,id
0,"京城最值得你来场文化之旅的博物馆保利集团,马未都,中国科学技术馆,博物馆,新中国",news_culture,0
1,发酵床的垫料种类有哪些？哪种更好？,news_culture,0
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture,0
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture,0
4,黄杨木是什么树？,news_culture,0
...,...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech,7
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military,8
382685,"又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！以色列,普京,俄罗斯,叙利亚,车臣",news_world,10
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world,10


#### 中文文本预处理

In [6]:
#数据清洗
#删除标点
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
#删除停用词
def stopwordslist():  
    stopwords = [line.strip() for line in open("chineseStopWords.txt", 'r', encoding='utf-8').readlines()]  
    return stopwords  
stopwords = stopwordslist()

In [7]:
df['clean_data'] = df['data'].apply(remove_punctuation)#删除标点
df['cut_data'] = df['clean_data'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))#分词
df

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LY\AppData\Local\Temp\jieba.cache
Loading model cost 0.943 seconds.
Prefix dict has been built successfully.


,data,label,id,clean_data,cut_data
0,"京城最值得你来场文化之旅的博物馆保利集团,马未都,中国科学技术馆,博物馆,新中国",news_culture,0,京城最值得你来场文化之旅的博物馆保利集团马未都中国科学技术馆博物馆新中国,京城 值得 来场 文化 之旅 博物馆 保利 集团 马未 中国科学技术馆 博物馆 新 中国
1,发酵床的垫料种类有哪些？哪种更好？,news_culture,0,发酵床的垫料种类有哪些哪种更好,发酵 床 垫料 种类 种 更好
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture,0,上联黄山黄河黄皮肤黄土高原怎么对下联,上联 黄山 黄河 黄皮肤 黄土高原 下联
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture,0,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣,林徽因 理由 拒绝 徐志摩 选择 梁思成 终身伴侣
4,黄杨木是什么树？,news_culture,0,黄杨木是什么树,黄杨木 树
...,...,...,...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech,7,A10处理器iPhoneSE二代值得期待吗,A10 处理器 iPhoneSE 二代 值得 期待
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military,8,先进战机叛逃将带来重大损失美军如何防止F22飞行员驾机叛逃,先进 战机 叛逃 带来 重大损失 美军 防止 F22 飞行员 驾机 叛逃
382685,"又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！以色列,普京,俄罗斯,叙利亚,车臣",news_world,10,又一国领导人放话只要普京下令数万大军碾压美国白宫以色列普京俄罗斯叙利亚车臣,一国 领导人 放 话 普京 下令 数万 大军 碾压 美国白宫 以色列 普京 俄罗斯 叙利亚 车臣
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world,10,如何看待美国总统连续撕毁美国签署的国际协议的举动,看待 美国 总统 连续 撕毁 美国 签署 国际 协议 举动


In [8]:
#划分训练集、测试集
X_train, X_test, y_train, y_test = train_test_split(df['cut_data'], df['id'], random_state = 0)

In [13]:
#用训练集生成词频向量和TF-IDF向量
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [14]:
#用朴素贝叶斯分类器MultinomialNB训练模型
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)
#用测试集生成F-IDF向量
X_test_tfidf = count_vect.transform(X_test)
y_pred = clf.predict(X_test_tfidf)

In [15]:
#评估结果
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
accuracy_score(y_pred, y_test)
print(classification_report(y_test, y_pred,target_names=id_df['label'].values))

                    precision    recall  f1-score   support

      news_culture       0.91      0.82      0.86      7044
news_entertainment       0.85      0.93      0.89      9901
       news_sports       0.95      0.95      0.95      9286
      news_finance       0.85      0.78      0.81      6869
        news_house       0.92      0.91      0.91      4410
          news_car       0.93      0.93      0.93      8975
          news_edu       0.88      0.91      0.89      6797
         news_tech       0.80      0.91      0.85     10464
     news_military       0.88      0.83      0.85      6185
       news_travel       0.84      0.81      0.83      5272
        news_world       0.81      0.82      0.82      6621
  news_agriculture       0.91      0.85      0.88      4791
         news_game       0.93      0.92      0.92      7460
             stock       0.00      0.00      0.00        97
        news_story       0.92      0.60      0.73      1500

          accuracy                    

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
